In [1]:
import random
import itertools
import csv
import copy 
from itertools import combinations, chain
from random import shuffle
import math
import sys
import numpy as np

In [2]:
class Student:
    def __init__(self, name, email, netid, preferences, skill):
        self.name = name
        self.email = email
        self.netid = netid
        self.preferences = preferences
        self.skill = skill

In [3]:
class PM:
    def __init__(self, name, email, netid, preferences):
        self.name = name
        self.email = email
        self.netid = netid
        self.preferences = preferences

In [4]:
def print_person(s):
    print("Name: " + s.name)
    print("Email: " + s.email)
    print("NetID: " + s.netid)
    print("Preferences: ", end="")
    print(s.preferences)
    print(" ")

In [5]:
# READ CSV #################################################

In [6]:
# preferences for students in order
# [Game dev, finance, education, music, cs for good, mental health, computer vision, AI/ML/DL, NLP, data analytics]
# create a mape where row idx correlates to preferences option in csv
pref_map = {}
pref_map[5] = "gd"
pref_map[6] = "fin"
pref_map[7] = "edu"
pref_map[8] = "mus"
pref_map[9] = "c4g"
pref_map[10] = "mh"
pref_map[11] = "cv"
pref_map[12] = "ai"
pref_map[13] = "nlp"
pref_map[14] = "da"

# list of all students
student_list = []

# read csv of students with preferences and populate student_list
with open('FA20_Project_Preference.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
#             print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
#             based on how the preferences are formatted in the csv, we only need three
#             options. Using the map we get the student's three options in the preferences
#             array in the form [1st, 2nd, 3rd]
            preferences = [0, 0, 0]
            for i in range(5, 15):
                try: 
                    if row[i][0] == "1":
                        preferences[0] = pref_map[i]
                    if row[i][0] == "2":
                        preferences[1] = pref_map[i]
                    if row[i][0] == "3":
                        preferences[2] = pref_map[i]
                except:
                    continue
            student_list.append(Student(row[2], row[1], row[3], preferences, row[17]))
            
            
            
# list of all PMs
pm_list = []

# read csv of pm with preferences and populate pm_list
with open('FA20_Project_Preference_PM.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
#             print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
#             based on how the preferences are formatted in the csv, we only need three
#             options. Using the map we get the pm's three options in the preferences
#             array in the form [1st, 2nd, 3rd]
            preferences = [0, 0, 0]
            for i in range(5, 15):
                try: 
                    if row[i][0] == "1":
                        preferences[0] = pref_map[i]
                    if row[i][0] == "2":
                        preferences[1] = pref_map[i]
                    if row[i][0] == "3":
                        preferences[2] = pref_map[i]
                except:
                    continue
            pm_list.append(PM(row[2], row[1], row[3], preferences))

In [7]:
############################################################

In [8]:
# SORTING ALGORITHM #################################################

In [9]:
print("PM Num: " + str(len(pm_list)))
print("Student Num: " + str(len(student_list)))
team_size_calculation = math.floor(len(student_list)/len(pm_list))
print("Students per PM (rounding down): " + str(team_size_calculation))

PM Num: 28
Student Num: 185
Students per PM (rounding down): 6


In [50]:
# due to the complexity finding an optimal grouping is unrealistic so we will match students to PMs who
# listed a preference that was in the student's top two preferences. 

# If we find an 'optimal' solution, i.e. all students matched up with a PM who matches their
# top two choices then we end the script and print that optimal grouping (highly unlikely unfortunately)
perfect_match = False

# min_displaced stores the minimum number of students left without a group after 10,000 iterations. 
# we then create a grouping with the min students displaced and can hand place the displaced students
min_displaced = len(student_list)+1
min_variation = sys.maxsize
num_iterations = 10000

# best_so_far will hold the grouping with the least displaced students AND lowest groups size variance
best_so_far = []

# if we find a perfect match before the iterations are done we will use that directly
while not perfect_match and num_iterations > 0:
    
    num_iterations -= 1
    
    # will store the current grouping attempt
    pm_groups = []
    
    # we will take a copy of the pm list and shuffle the elements for this grouping attempt
    pm_list_test = pm_list.copy()
    shuffle(pm_list_test)
    
    # we will take a copy of the student list and shuffle the elements for this grouping attempt
    student_list_test = student_list.copy()
    shuffle(student_list_test)
    
    # if a student is in a group that fits their preferences they are deleted from circulation and added
    # to the placed set
    student_placed = set()
    
    # create groups for each pm
    for pm in pm_list_test:
        
        # curent pms group attempt
        curr_group = []
        
        # we want each group to be a max of math.ceil(len(student_list)/len(pm_list))
        num_members_needed = team_size_calculation
        
        # we find the next student who would fit this PM from the randomized list
        for student in student_list_test:
            for i in range(0, 2):
                # if the student's top two preferences fit the PM we add them to that group
                if student.preferences[i] in pm.preferences[:3]:
                    # add to pms group, add to placed set, remove from circulation
                    curr_group.append(student)
                    student_placed.add(student)
                    student_list_test.remove(student)
                    # team needs one less person now
                    num_members_needed -=1
                    break
            # if team is at max size we are done with this pm
            if num_members_needed == 0:
                break
        curr_group.insert(0, pm)
        pm_groups.append(curr_group)

    perfect_match = True
    displaced = 0
    for student in student_list:
        if student not in student_placed:
            # if any student is displaced then ths is not a perfect match
            perfect_match = False
            displaced += 1
     
    # update best grouping so far 
    if min_displaced > displaced:
        best_so_far = pm_groups
        min_displaced = displaced
    # if min_displaced is equal then pick the one with lower variance
    elif min_displaced == displaced:
        group_sizes = []
        for g in pm_groups:
            group_sizes.append(len(g))
        variance = np.var(group_sizes)
        if min_variation > variance:
            best_so_far = pm_groups
            min_variation = variance


# PRINT RESULTS
print("Num min displaced: ", end="")
print(min_displaced)

student_placed = set()

for i in range(len(best_so_far)):
    for s in best_so_far[i]:
        student_placed.add(s)

for s in student_list:
    if s not in student_placed:
        print(s.name, end=", ")
        print(s.netid, end=", ")
        print(s.preferences, end=", ")
        print("Coding skill level: " + str(s.skill))

        
for i in range(len(best_so_far)):
    print("#####################")
    print("Groups size: " + str(len(best_so_far[i])-1) + "/" + str(team_size_calculation))
    print("PM Name: ", end = "")
    for s in best_so_far[i]:
        print(s.name, end=", ")
        print(s.netid, end=", ")
        print(s.preferences, end=", ")
        try:
            print("Coding skill level: " + str(s.skill))
        except:
            print("\n")

Num min displaced: 17
Akaash Kolachina, akaashk2, ['ai', 'c4g', 'fin'], Coding skill level: 2
Aniket Tripathy, anikett2, ['mus', 'c4g', 'gd'], Coding skill level: 3
Carlos Conley, cconley3, ['gd', 'da', 'ai'], Coding skill level: 4
Chirag Mehta, chiragm3, ['ai', 'mus', 'da'], Coding skill level: 4
Erika Huerta, ehuerta2, ['mh', 'c4g', 'edu'], Coding skill level: 3
Gabriel Aaron, gaaron2, ['ai', 'fin', 'da'], Coding skill level: 3
Gagan Kadadevarmath, gagan2, ['ai', 'gd', 'cv'], Coding skill level: 2
Jackson Kennel, kennel2, ['nlp', 'cv', 'ai'], Coding skill level: 2
Jonah Tjandra, jonahlt2, ['ai', 'cv', 'gd'], Coding skill level: 3
Jordan, jzalwan2, ['da', 'ai', 'cv'], Coding skill level: 2
Lisa Liu, lisaliu2, ['ai', 'cv', 'gd'], Coding skill level: 2
Michal Juscinksi, michalj2, ['da', 'gd', 'ai'], Coding skill level: 3
Priya Kumar, priyak5, ['ai', 'cv', 'nlp'], Coding skill level: 3
Raghav, rsanand2, ['ai', 'gd', 'mus'], Coding skill level: 3
Salman Shaheen, salmans3, ['c4g', 'cv', 'm